### Environment Setup

In [7]:
import os
import pandas as pd
import requests
from dotenv import load_dotenv

load_dotenv()
API_KEY_CMC = os.getenv("CMC_API_KEY")
API_KEY_BINANCE = os.getenv('API_KEY_BINANCE')
API_SECRET_BINANCE = os.getenv('API_SECRET_BINANCE')

if API_KEY_CMC:
    print("API_KEY_CMC found!")
else:
    print("API_KEY_CMC not found. Please set it in the .env file. See README.md for more info.")

API_KEY_CMC found!


### Data Fetching

In [8]:
# Fetch categories data from CoinMarketCap API
categories_url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/categories"
headers = {"Accepts": "application/json", "X-CMC_PRO_API_KEY": API_KEY_CMC}

response = requests.get(categories_url, headers=headers)

if response.status_code != 200:
    print(f"Error fetching categories: {response.status_code}")
    exit()

# Parse response data
categories_data = response.json()["data"]
print(f"Number of categories: {len(categories_data)}")

Number of categories: 260


### Data Processing

In [12]:
# Convert categories data to a DataFrame
categories_df = pd.DataFrame(categories_data)

# Rank market_cap_change and volume_change
categories_df["market_cap_rank"] = categories_df["market_cap_change"].rank(ascending=False)
categories_df["volume_rank"] = categories_df["volume_change"].rank(ascending=False)

# Combine ranks (lower is better)
# categories_df["combined_rank"] = categories_df["market_cap_rank"] + categories_df["volume_rank"]

# Sort by combined rank (lower combined_rank indicates stronger performance)
categories_df = categories_df.sort_values(by="volume_rank")
# print(categories_df[["name", "combined_rank", "market_cap_rank", "volume_rank"]])

# Get the top 1 category
top_category = categories_df.iloc[0]
print(f"Top category: {top_category['name']} (volume rank: {top_category['volume_rank']})")
# print(f"Top category: {top_category['name']} (combined rank: {top_category['combined_rank']})")

Top category: Ethereum Ecosystem (volume rank: 1.0)


In [4]:
# Fetch coins in the top category
category_id = top_category["id"]
category_name = top_category["name"]

coins_url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/category"
headers = {"Accepts": "application/json", "X-CMC_PRO_API_KEY": API_KEY_CMC}
params = {"id": category_id}

response = requests.get(coins_url, headers=headers, params=params)

# Handle API response
if response.status_code != 200:
    print(f"Error: {response.status_code}, Message: {response.text}")
    exit()

data = response.json().get("data", {}).get("coins", [])
if not data:
    print(f"No coins found in category '{category_name}'.")
    exit()

# Process coins data
coins_df = pd.DataFrame(data)
if "quote" in coins_df.columns:
    coins_df["market_cap"] = coins_df["quote"].apply(
        lambda x: x.get("USD", {}).get("market_cap") if isinstance(x, dict) else None
    )

# Convert market_cap to numeric and sort
coins_df["market_cap"] = pd.to_numeric(coins_df["market_cap"], errors="coerce")
# top_coins = coins_df.sort_values(by="market_cap", ascending=False).head(5)

# # Print top 5 coins
# print(f"Top 5 coins in category '{category_name}':")
# print(top_coins[["name", "market_cap", "symbol"]])

# print top 1 coin
top_coin = coins_df.iloc[0]
print(f"Top coin in category '{category_name}':")
print(top_coin[["name", "market_cap", "symbol"]])

Top coin in category 'Filesharing':
name                   Filecoin
market_cap    3563114353.569911
symbol                      FIL
Name: 0, dtype: object


In [ ]:
import requests

api_key = API_KEY_CMC
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/info'
parameters = {
    'symbol': 'BTC,ETH'
}
headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': api_key,
}

response = requests.get(url, headers=headers, params=parameters)
data = response.json()

# 輸出比特幣的分類標籤
print('Bitcoin (BTC) tags:', data['data']['BTC']['category'])

# 輸出以太坊的分類標籤
print('Ethereum (ETH) tags:', data['data']['ETH']['tags'])


Bitcoin (BTC) tags: ['mineable', 'pow', 'sha-256', 'store-of-value', 'state-channel', 'coinbase-ventures-portfolio', 'three-arrows-capital-portfolio', 'polychain-capital-portfolio', 'binance-labs-portfolio', 'blockchain-capital-portfolio', 'boostvc-portfolio', 'cms-holdings-portfolio', 'dcg-portfolio', 'dragonfly-capital-portfolio', 'electric-capital-portfolio', 'fabric-ventures-portfolio', 'framework-ventures-portfolio', 'galaxy-digital-portfolio', 'huobi-capital-portfolio', 'alameda-research-portfolio', 'a16z-portfolio', '1confirmation-portfolio', 'winklevoss-capital-portfolio', 'usv-portfolio', 'placeholder-ventures-portfolio', 'pantera-capital-portfolio', 'multicoin-capital-portfolio', 'paradigm-portfolio', 'bitcoin-ecosystem', 'ftx-bankruptcy-estate', '2017-2018-alt-season']
Ethereum (ETH) tags: ['pos', 'smart-contracts', 'ethereum-ecosystem', 'coinbase-ventures-portfolio', 'three-arrows-capital-portfolio', 'polychain-capital-portfolio', 'binance-labs-portfolio', 'blockchain-capit